In [1]:
# Cell 1 — Imports & config
import pandas as pd
import re
from pathlib import Path

# Path to your orders CSV
CSV_PATH = Path("../doc/orders.csv")

# Output .sql file for INSERTs (optional)
OUTPUT_SQL = Path("bills_inserts.sql")

# Helper: SQL quote and NULL handling
def sql_quote(val):
    if val is None:
        return "NULL"
    s = str(val)
    # treat empty strings as NULL
    if s.strip() == "":
        return "NULL"
    # escape single quotes
    s = s.replace("'", "''")
    return f"'{s}'"

def is_valid_sg_phone(s):
    # exactly 8 digits, all numeric
    return bool(re.fullmatch(r"\d{8}", s or ""))


In [7]:
# Cell 2 — Load CSV into DataFrame
df = pd.read_csv(CSV_PATH)

# Ensure expected columns exist (will raise if missing)
expected = ["Date","Time","Order","Payment","Card","CardType","Item","TotalPrice","Phone","Firstname","Lastname","Staff"]
missing = [c for c in expected if c not in df.columns]
if missing:
    raise ValueError(f"CSV missing columns: {missing}")

df.head()


,Date,Time,Order,Payment,Card,CardType,Item,TotalPrice,Phone,Firstname,Lastname,Staff
0,2024-03-01,10:15:51,20240301001,card,3742-8375-6443-8590,americanexpress,Rendang,4.0,NaN,NaN,NaN,STAFF-01
1,2024-03-01,12:19:23,20240301002,card,5108-7574-2920-6803,mastercard,Ayam Balado,14.0,93627414.0,Ignazio,Abrahmer,STAFF-03
2,2024-03-01,12:19:23,20240301002,card,5108-7574-2920-6803,mastercard,Ayam Balado,14.0,93627414.0,Ignazio,Abrahmer,STAFF-03
3,2024-03-01,12:19:23,20240301002,card,5108-7574-2920-6803,mastercard,Ayam Balado,14.0,93627414.0,Ignazio,Abrahmer,STAFF-03
4,2024-03-01,12:19:23,20240301002,card,5108-7574-2920-6803,mastercard,Ayam Balado,14.0,93627414.0,Ignazio,Abrahmer,STAFF-04


In [ ]:
# Cell 3 — Select requested columns and DROP duplicates by Order (keep first)
cols_needed = ["Date","Time","Order","Card","CardType","TotalPrice","Phone"]
df_sel = df[cols_needed].copy()

# Drop duplicates by Order, keeping the first occurrence
df_uniq = df_sel.drop_duplicates(subset=["Order"], keep="first").reset_index(drop=True)

# Basic normalization
# - Strip whitespace
for c in ["Card","CardType"]:
    df_uniq[c] = df_uniq[c].astype(str).str.strip()

# Correctly clean the 'Phone' column
df_uniq["Phone"] = df_uniq["Phone"].apply(lambda x: str(int(x)) if pd.notna(x) and str(x).replace('.', '', 1).isdigit() else None)

# Convert TotalPrice to numeric (coerce invalid -> NaN -> treat as 0.0)
df_uniq["TotalPrice"] = pd.to_numeric(df_uniq["TotalPrice"], errors="coerce").fillna(0.0)

df_uniq.head()

,Date,Time,Order,Card,CardType,TotalPrice,Phone
0,2024-03-01,10:15:51,20240301001,3742-8375-6443-8590,americanexpress,4.0,nan
1,2024-03-01,12:19:23,20240301002,5108-7574-2920-6803,mastercard,14.0,93627414.0
2,2024-03-01,13:46:33,20240301003,3466-5960-1418-4580,americanexpress,9.0,nan
3,2024-03-01,13:48:15,20240301004,3379-4110-3466-1310,americanexpress,8.0,nan
4,2024-03-01,15:39:48,20240301005,3742-8382-6101-0570,americanexpress,3.5,89007281.0


In [11]:
# Cell 4 — Derive fields for bills INSERTs
# bills(bill_date, bill_time, bill_id, payment, total_bill, card_number, card_type, phone)

def infer_payment(card):
    return "card" if (card is not None and str(card).strip() != "") else "cash"

rows = []
for _, r in df_uniq.iterrows():
    bill_date   = r["Date"]           # yyyy-mm-dd
    bill_time   = r["Time"]           # hh:mm:ss
    bill_id     = str(r["Order"])     # text id
    card_number = r["Card"] if str(r["Card"]).strip() != "" else None
    card_type   = r["CardType"] if str(r["CardType"]).strip() != "" else None
    total_bill  = float(r["TotalPrice"])
    
    # Corrected phone number extraction
    phone_raw = r["Phone"]
    
    # Check if the phone number is a float and not NaN
    if pd.isna(phone_raw):
        phone_cleaned = None
    else:
        try:
            # Convert the float to an integer to remove the .0, then to a string
            # This handles cases like 93627414.0
            phone_cleaned = str(int(phone_raw))
        except (ValueError, TypeError):
            # Fallback for non-numeric or malformed data
            phone_cleaned = str(phone_raw).strip()
    
    # Validate the cleaned phone number
    phone = phone_cleaned if is_valid_sg_phone(phone_cleaned) else None

    payment = infer_payment(card_number)

    rows.append({
        "bill_date": bill_date,
        "bill_time": bill_time,
        "bill_id":   bill_id,
        "payment":   payment,
        "total_bill": total_bill,
        "card_number": card_number,
        "card_type":   card_type,
        "phone":       phone
    })

bills_df = pd.DataFrame(rows)
bills_df.head()

,bill_date,bill_time,bill_id,payment,total_bill,card_number,card_type,phone
0,2024-03-01,10:15:51,20240301001,card,4.0,3742-8375-6443-8590,americanexpress,None
1,2024-03-01,12:19:23,20240301002,card,14.0,5108-7574-2920-6803,mastercard,None
2,2024-03-01,13:46:33,20240301003,card,9.0,3466-5960-1418-4580,americanexpress,None
3,2024-03-01,13:48:15,20240301004,card,8.0,3379-4110-3466-1310,americanexpress,None
4,2024-03-01,15:39:48,20240301005,card,3.5,3742-8382-6101-0570,americanexpress,None


In [12]:
# Cell 5 — Generate PostgreSQL INSERT statements for `bills`
# Assumes your table definition:
# bills(bill_date DATE, bill_time TIME, bill_id VARCHAR(11) PK, payment VARCHAR(4), total_bill NUMERIC,
#       card_number VARCHAR(19), card_type VARCHAR(100), phone VARCHAR(8))

inserts = []
for _, r in bills_df.iterrows():
    stmt = (
        "INSERT INTO bills VALUES ("
        f"{sql_quote(r['bill_date'])}, "
        f"{sql_quote(r['bill_time'])}, "
        f"{sql_quote(r['bill_id'])}, "
        f"{sql_quote(r['payment'])}, "
        f"{r['total_bill']}, "  # numeric (no quotes)
        f"{sql_quote(r['card_number'])}, "
        f"{sql_quote(r['card_type'])}, "
        f"{sql_quote(r['phone'])}"
        ");"
    )
    inserts.append(stmt)

# Preview first few inserts
print("\n".join(inserts[:5]))


INSERT INTO bills VALUES ('2024-03-01', '10:15:51', '20240301001', 'card', 4.0, '3742-8375-6443-8590', 'americanexpress', NULL);
INSERT INTO bills VALUES ('2024-03-01', '12:19:23', '20240301002', 'card', 14.0, '5108-7574-2920-6803', 'mastercard', NULL);
INSERT INTO bills VALUES ('2024-03-01', '13:46:33', '20240301003', 'card', 9.0, '3466-5960-1418-4580', 'americanexpress', NULL);
INSERT INTO bills VALUES ('2024-03-01', '13:48:15', '20240301004', 'card', 8.0, '3379-4110-3466-1310', 'americanexpress', NULL);
INSERT INTO bills VALUES ('2024-03-01', '15:39:48', '20240301005', 'card', 3.5, '3742-8382-6101-0570', 'americanexpress', NULL);


In [18]:
# Cell 6 — (Optional) Save all INSERTs to a .sql file
OUTPUT_SQL.write_text("\n".join(inserts), encoding="utf-8")
OUTPUT_SQL


WindowsPath('bills_inserts.sql')